## EGU Topic Modeling and prediction

This notebook uses topic modeling to analyze the EGU conference using the abstracts submitted for the years 2011 to 2018. The notebook can be used to detect trends and visualize topics along these years using one or many categories. The abstracts were parsed into text and ingested into a Apache Solr instance.

We parsed the PDFs using PDFMiner's utility pdf2txt

```sh
ls *.pdf | xargs -n1 -P8 bash -c 'pdf2txt.py -o output/$0.txt -t text $0'
```

**Solr Schema**
 
 
 
```json
doc = {
"entities":[
    "Jeffrey Obelcz  and Warren T. Wood",
    "NRC Postdoctoral Fellow",
    "Naval Research Lab",
    "Seaﬂoor Sciences",
    "United States jbobelcz@gmail.com",
    "Naval",
    "Research Lab",
    "Seaﬂoor Sciences",
    "United States"],
"id": "EGU2018-9778",
"sessions": ["ESSI4.3"],
"file": ["EGU2018-9778"],
"presentation": ["Posters"],
"year": [2018],
"title": ["Towards a Quantitative Understanding of Parameters Driving Submarine Slope Failure: A Machine Learning Approach"],
"category": ["ESSI"],
"abstract":["Submarine slope failure is a ubiquitous process and dominant pathway for sediment and organic carbon ﬂux from continental margins to the deep sea. Slope failure occurs over a wide range of temporal and spatial scales ..."]
}
```

Example:

```json
doc = {
"entities":[
    "Jeffrey Obelcz  and Warren T. Wood",
    "NRC Postdoctoral Fellow",
    "Naval Research Lab",
    "Seaﬂoor Sciences",
    "United States jbobelcz@gmail.com",
    "Naval",
    "Research Lab",
    "Seaﬂoor Sciences",
    "United States"],
"id": "EGU2018-9778",
"sessions": ["ESSI4.3"],
"file": ["EGU2018-9778"],
"presentation": ["Posters"],
"year": [2018],
"title": ["Towards a Quantitative Understanding of Parameters Driving Submarine Slope Failure: A Machine Learning Approach"],
"category": ["ESSI"],
"abstract":["Submarine slope failure is a ubiquitous process and dominant pathway for sediment and organic carbon ﬂux from continental margins to the deep sea. Slope failure occurs over a wide range of temporal and spatial scales ..."]
}
```

## Application

This notebook can be used to analyze what a corpora of scientific text talks about, in this case we used EGU abstracts but it can be used on any text corpora.

The topics are examined using LDAVis, which displays the topics on an X-Y plot (intertopic distance map). Topic are represented by circles whose areas are proportional to the relative prevalences of the topics in the corpus. In the display the user can enter a topic number (note 1-base numbering vs. 0-base numbering in Topic List below cells 5 and 7); the terms are displayed on the right, ranked by significance (weight). A topic can be selected on the fly by hovering over its circle; clicking selects that topic. A user can also click on a term in the RH panel to show the topics in which that term occurs. 

The slider at the top of the RH panel allows the user to vary the “saliency”, i.e. uniqueness of the terms to that topic. A value of 0.6 is optimal, according to the authors of the algorithm. Blue bars represent overall term frequency while red bars show term frequency within the topic, which will be different when the saliency Is selected be less than one.

Please see the annotated image of the LDAVis display.

![](https://raw.githubusercontent.com/USCDataScience/pdi-topics/master/notebooks/pdi/img/pylda.png)



The final cell of this notebook lists the titles and IDs of abstracts belonging to a specified topic.

* category: the Program Group, e.g. CL, AS etc. Keep in mind that these codes have changed through the years.
* presentation: Presentation type: oral, poster, pico etc.
* session: the session that the abstract appeared in; sessions may be co-organized.

-----
EGU Programme Groups:

### Disciplinary Sessions 

Geodynamics (GD) -
Geosciences Instrumentation & Data Systems (GI) -
Geomorphology (GM) -
Geochemistry, Mineralogy, Petrology & Volcanology (GMPV) -
Hydrological Sciences (HS) -
Natural Hazards (NH) -
Nonlinear Processes in Geosciences (NP) -
Ocean Sciences (OS) -
Planetary & Solar System Sciences (PS) -
Seismology (SM) -
Stratigraphy, Sedimentology & Palaeontology (SSP) -
Soil System Sciences (SSS) -
Solar-Terrestrial Sciences (ST) -
Tectonics & Structural Geology (TS) -
Atmospheric Sciences (AS) -
Biogeosciences (BG) -
Climate: Past, Present, Future (CL) -
Cryospheric Sciences (CR) -
Earth Magnetism & Rock Physics (EMRP) -
Energy, Resources and the Environment (ERE) -
Earth & Space Science Informatics (ESSI) -
Geodesy (G) -
Geodynamics (GD) -
Geosciences Instrumentation & Data Systems (GI) -
Geomorphology (GM) -
Geochemistry, Mineralogy, Petrology & Volcanology (GMPV) -
Hydrological Sciences (HS) -
Natural Hazards (NH) -
Nonlinear Processes in Geosciences (NP) -
Ocean Sciences (OS) -
Planetary & Solar System Sciences (PS) -
Seismology (SM) -
Stratigraphy, Sedimentology & Palaeontology (SSP) -
Soil System Sciences (SSS) -
Solar-Terrestrial Sciences (ST) -
Tectonics & Structural Geology (TS) -

### Union Sessions

Union Symposia (US)
Great Debates (GDB)
Medal Lectures (ML)
Short courses (SC)
Educational and Outreach Symposia (EOS)
EGU Plenary, Ceremonies and Networking (PCN)
Feedback and administrative meetings (FAM)
Townhall and splinter meetings (TSM)
Side events (SEV)
Press conferences (PC)

#### Interdisciplinary Events (IE)


In [ ]:
# Cell 1: Import requirements
import urllib
import json
import string
import pandas as pd

# NLP libraries
import spacy
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import Phrases
from gensim.models.phrases import Phraser

# We are using spacy as a parser so we disable their other capabilities to speed up things
nlp = spacy.load('en', disable=['tagger', 'ner'])


In [ ]:
# Cell 2: Querying Solr

# terms = ['ice', 'climate'] to include only abstracts with specified terms
terms = ['*']
years = ['2011', '2018']
entities = ['*']
sessions = ['CR','NH']

# Return "page_size" documents with each Solr query until complete
page_size = 1000
cursorMark = '*'

solr_documents = []
solr_root = 'http://integration.pdi-solr.labs.nsidc.org/solr/egu/select?indent=on&'
more_results = True


if terms[0] != '*':
    terms_wirldcard = ['*' + t + '*' for t in terms]
else:
    terms_wirldcard = ['*']
    
if sessions[0] != '*':
    sessions_wirldcard = ['*' + s + '*' for s in sessions]
else:
    sessions_wirldcard = ['*']
    
if entities[0] != '*':
    entities_wirldcard = ['*' + e + '*' for e in entities]
else:
    entities_wirldcard = ['*']

terms_query = '%20OR%20abstract:'.join(terms_wirldcard)
years_query = '%20OR%20year:'.join(years)  
entities_query = '%20OR%20entities:'.join(entities_wirldcard)
sessions_query = '%20OR%20sessions:'.join(sessions_wirldcard)
query_string = 'q=(abstract:{}%20AND%20abstract:/.{{2}}.*/%20AND%20NOT%20title:/.{{300}}.*/)%20AND%20(year:{})' + \
                '%20AND%20(entities:{})%20AND%20(sessions:{})&wt=json&rows={}&cursorMark={}&sort=id+asc'
while (more_results):    
    solr_query = query_string.format(terms_query,
                                     years_query,
                                     entities_query,
                                     sessions_query,
                                     page_size,
                                     cursorMark)
    solr_url = solr_root + solr_query
    print('Querying: \n' + solr_url)
    req = urllib.request.Request(solr_url)
    # parsing response
    r = urllib.request.urlopen(req).read()
    json_response = json.loads(r.decode('utf-8'))
    solr_documents.extend(json_response['response']['docs'])
    nextCursorMark = json_response['nextCursorMark']
    if (nextCursorMark == cursorMark):
        more_results = False
        break
    else: 
        cursorMark = nextCursorMark

total_found = json_response['response']['numFound']
print("Processing {0} out of {1} total. \n".format(len(solr_documents), total_found))

In [ ]:
%%time
# Cell 3: Cleaning our documents 

# Helper function
def flatten(top_list):
    for inner in top_list:
        if isinstance(inner, (list,tuple)):
            for j in flatten(inner):
                yield j
        else:
            yield inner


years = {str(year) for year in range(2000,2020)}

nlp.Defaults.stop_words |= years
nlp.Defaults.stop_words |= ENGLISH_STOP_WORDS

nlp.Defaults.stop_words |= {
                            'area', 'data', 'event', 'use', 'group', 'research', 'model',
                            'metadata', 'content', 'sharing', 'previous', 'http','study',
                            'datum', 'result', 'different', 'change'
                           }

# After we generate the bigrams there are terms that we might want to remove including bigrams.
post_process_stopwords = {'sea_ice', 'water'}

ALL_STOP_WORDS = ENGLISH_STOP_WORDS.union(nlp.Defaults.stop_words)

# Function to clean up the documents, lematizes the words to their regular form and removes the stop words.
def clean_sentences(doc):
    doc = nlp(doc)
    processed_sentences = []
    for num, sentence in enumerate(doc.sents):
        tokens = [token.lemma_.encode('ascii',errors='ignore').decode().strip().lower() for token in sentence if token.lemma_ not in string.punctuation]
        cleaned_sentence = [token for token in tokens if token not in ALL_STOP_WORDS]
        cleaned_sentence = [token for token in cleaned_sentence if token != '-PRON-']
        processed_sentences.append(cleaned_sentence)
    return processed_sentences

# document list will contain our corpus after cleaning it.
gensim_unigram_documents = []
document_list = []
# artifact of parsing the sessions from the pdf documents
garbage_str = '</a>'

for doc in solr_documents:
    abstract = clean_sentences(doc['abstract'][0])
    if 'sessions' in doc:
        sindex = doc['sessions'][0].find(garbage_str)
        if sindex != -1:
            sessions = doc['sessions'][0][0:sindex]
        else: 
            sessions = doc['sessions'][0]
    else:
        sessions = 'NAN'
    document_list.append({ 'id': doc['id'],
                                   'text': abstract, 
                                   'year': str(doc['year'][0]),
                                   'title': doc['title'][0],
                                   'category': doc['category'][0].replace('/', '').replace('<', ''),
                                   'sessions':sessions})
    
    gensim_unigram_documents.append(abstract)


In [ ]:
# Cell #4 creating the bigram corpus

# unigram sentences is going to be used to train our bigram phraser
unigram_sentences = []
# bigram corpus will contain an array of documents and their tokens, with bigram tokens included
bigram_corpus = []
bigram_docs = []

for doc in gensim_unigram_documents:
    for sentence in doc:
        unigram_sentences.append(sentence)

bigram_model = Phrases(unigram_sentences, min_count=2)
bigram_phraser = Phraser(bigram_model)

for unigram_doc in gensim_unigram_documents:
    bigram_sentences = []
    for unigram_sentence in unigram_doc:
        bigram_sentence = ' '.join(bigram_phraser[unigram_sentence])
        bigram_sentences.append(bigram_sentence)
    bigram_docs.append(bigram_sentences)


for bigram_sentdoc in bigram_docs:
    bigram_tokens = []
    bigram_doc = []
    for sentence in bigram_sentdoc:
        bigrams = sentence.split()
        bigram_tokens.append([b for b in bigrams if b not in post_process_stopwords])
    bigram_corpus.append(list(flatten(bigram_tokens)))

df = pd.DataFrame.from_dict(document_list)
axis_category = pd.DataFrame(df.groupby(['category', 'year'])['category'].count()).rename(columns={'category': 'count'})
print(axis_category)

### Building the LDA model using Gensim a library for topic modeling, the output is a list of topics present in our corpus.

In [ ]:
# Cell 5: Using GENSIM to do topic modelling, this cell takes some time... hang on.

# num pases should be adjusted, 3 is just a guesstimate of when convergence will be achieved.
num_passes = 3
num_topics = 20
words_per_topic = 9
print_topics = False

dictionary = corpora.Dictionary(bigram_corpus)
lda_corpus = [dictionary.doc2bow(text) for text in bigram_corpus]

lda_model = LdaMulticore(lda_corpus,
                         num_topics=num_topics,
                         id2word=dictionary,
                         passes=num_passes,
                         workers=2
                        )

topics = lda_model.print_topics(num_topics=num_topics, num_words=words_per_topic)
if print_topics:
    print ("Topic List: \n")
    for topic in topics:
        t = str((int(topic[0])+ 1))
        print('Topic ' + t + ': ', topic[1:])

import warnings
warnings.filterwarnings('ignore')

import pyLDAvis.gensim
print ("\nPyLDAVis: \n")
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(corpus=lda_corpus,
                        topic_model=lda_model,
                        dictionary=dictionary,
                        sort_topics=False)


### Now that we have a trained model we can classify a new unseen document.

In [ ]:
# Cell 6: Listing papers containing a particular ngram

# use unigrams, sea_ice becomes sea, ice etc
terms = set(['social','resilience'])

top_n = 10

def createLink(doc):
    baseURL = 'https://meetingorganizer.copernicus.org/EGU' + str(doc['year']) + '/' + doc['id'] + '.pdf'
    return baseURL

# Ode to Python's comprehension lists
matches  = [doc for doc in document_list if terms == set(flatten(doc['text'])).intersection(terms)]

from IPython.core.display import display, HTML
# Let's predict the first 10 documents
for doc in matches[0:top_n]:
    display(HTML('<br>Abstract <a href="{}" target="_blank">{}</a> '.format(
        createLink(doc),
        doc['id'])))    


In [ ]:
# Cell 7: Classifying an unseen document using our GENSIM model

# For practical purposes we use a mocked up document but we can easily query Solr or another store to get the content we want to classify
# Eventually all this should be served in as a web service 
# taken from https://meetingorganizer.copernicus.org/EGU2018/EGU2014-2415.pdf

unseen_document = """
Waves  in  the  Southern  Ocean  are  the  largest  in  the  planet.  In  the  Southern  Hemisphere,  the  absence  of  large
landmasses at high latitudes allows the wind to feed energy into the ocean over a virtually unlimited fetch. The
enormous amount air-sea momentum exchanged over the Southern Ocean plays a substantial role on the global
climate. However, large biases affect the estimation of wave regime around the Antarctic continent making climate
prediction susceptible to uncertainty.
 """

parsed_doc = list(flatten(clean_sentences(unseen_document)))
vec = dictionary.doc2bow(parsed_doc)
predicted_topics = lda_model[vec]
predicted_topics = [(p[0]+1, p[1]) for p in predicted_topics]
print(predicted_topics)